# Project 3: What makes a top movie?

In [ ]:
from bs4 import BeautifulSoup
import pandas as pd 
import seaborn as sns
import requests
import re


Data comes from the movies [here](https://www.imdb.com/chart/top/?ref_=nv_mv_250).

In [ ]:
#Basic exersize: Go through each movie and get the title
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36'}
title_page = requests.get('https://www.imdb.com/chart/top/?ref_=nv_mv_250', headers=headers)
tp_soup = BeautifulSoup(title_page.content, 'html.parser')


In [ ]:

top_movies = tp_soup.find_all('li', attrs={'class': 'ipc-metadata-list-summary-item sc-1364e729-0 caNpAE cli-parent'})

titles = [re.sub(r'\d+\.\s*','' ,str(item.find('h3', attrs={'class':'ipc-title__text'}).contents[0])) for item in top_movies]
display(titles)


In [ ]:
movie_page_urls = ['https://www.imdb.com' + str(item.find('a').get('href')[0:16] + '/reviews') for item in top_movies]
movie_page_urls


In [ ]:
all_urls = []
for url in movie_page_urls:
    for i in range(1, 11):
        all_urls.append(url +"?sort=userRating&dir=desc&ratingFilter=" + str(i))
all_urls

In [ ]:
# Each index of all_pages_reviews_list is the top voted reviews of a given rating. 

import time
all_pages_reviews_list = []
for i in all_urls:
    html_reviews = requests.get(i, headers=headers)
    soup_reviews = BeautifulSoup(html_reviews.text)
    all_pages_reviews = soup_reviews.find_all('div', attrs={'class': 'text show-more__control'})
    all_pages_reviews = [i.get_text(strip=True ) for i in all_pages_reviews]
    all_pages_reviews_list.append(all_pages_reviews)
    time.sleep(10)
display(len(all_pages_reviews_list))

#10 items per movie
#about 20 reviews per item
# dict: movie[all reviews of movie]

In [ ]:
all_pages_reviews_list[20]

In [ ]:
movie_dict = {}
for i in range(0, len(all_pages_reviews_list)):
    movie_dict[titles[i//10] + ' ' + str(i%10 + 1)] = all_pages_reviews_list[i]
movie_dict

In [ ]:
for item in movie_dict:
    print(item, ' ', len(movie_dict[item]))

In [ ]:
df = pd.DataFrame(dict([(key, pd.Series(value)) for key, value in movie_dict.items()]))
#df.to_csv('review_csv.csv')
df

In [ ]:
by_reviews_dict = {}

for i in range(len(all_pages_reviews_list)):
    if i%10 + 1 in by_reviews_dict.keys():
        by_reviews_dict[i%10 +1].append(all_pages_reviews_list[i])
    else:
        by_reviews_dict[i%10 + 1] = []
        by_reviews_dict[i%10 +1].append(all_pages_reviews_list[i])

review_num_df = pd.DataFrame(dict([(key, pd.Series(value)) for key, value in by_reviews_dict.items()]))
review_num_df.to_csv('columns_as_rating.csv')